In [6]:
def convert_bronze_to_silver():
    from pyspark.sql import SparkSession    
    from pyspark.sql.functions import col, concat_ws, to_timestamp, current_timestamp, substring
    import psycopg2
    import datetime

    nm_processo = "bronze_to_silver_posicao"
    dt_delta = datetime.date.today()

    # === Spark Session ===
    spark = (
        SparkSession
        .builder
        .appName('pipeline_sptrans')
        .config('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension')
        .config('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog')
        .config("spark.hadoop.fs.s3a.access.key", "datalake")
        .config("spark.hadoop.fs.s3a.secret.key", "datalake")
        .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .getOrCreate()
    )

    # === PostgreSQL Connection Properties ===
    postgres_url = "jdbc:postgresql://db:5432/db"
    properties = {
        "user": "admin",
        "password": "admin",
        "driver": "org.postgresql.Driver"
    }

    # Conexão psycopg2 para log
    conn = psycopg2.connect(
        host="db",
        port=5432,
        database="db",
        user="admin",
        password="admin"
    )
    cur = conn.cursor()

    try:
        # === Lê última dt_delta do Silver ===
        try:
            df_silver_max = spark.read.jdbc(
                url=postgres_url,
                table="db.silver_sptrans.posicao",
                properties=properties
            )
            max_dt = df_silver_max.agg({"dt_delta": "max"}).collect()[0][0]
        except:
            max_dt = None

        # === Lê Bronze ===
        df_bronze = spark.read \
            .format("jdbc") \
            .option("url", postgres_url) \
            .option("dbtable", "db.bronze_sptrans.posicao") \
            .option("user", properties["user"]) \
            .option("password", properties["password"]) \
            .load()

        # === Transformações para Silver ===
        df_silver = df_bronze.select(
            col("hr").cast("string"),
            col("c").cast("string"),
            col("cl").cast("int"),
            col("sl").cast("int"),
            col("lt0").cast("string"),
            col("lt1").cast("string"),
            col("qv").cast("int"),
            col("p").cast("int"),      # veículo único
            col("a").cast("boolean"),
            col("ta").cast("string"),
            col("py").cast("double"),
            col("px").cast("double"),
            col("sv").cast("string"),
            col("vs_is").cast("string"),
            to_timestamp(
                concat_ws(" ", substring(col("dt_processamento"), 1, 10), col("hr")),
                "yyyy-MM-dd HH:mm"
            ).alias("dt_delta"),
            current_timestamp().alias("dt_processamento")  # timestamp da carga
        )

        # === Filtra só novos dt_delta ===
        if max_dt is not None:
            df_silver = df_silver.filter(col("dt_delta") > max_dt)

        # === Remove duplicados (veículo p + instante dt_delta) ===
        df_silver = df_silver.dropDuplicates(["p", "dt_delta"])

        # Conta quantidade de registros
        qtde_registro = df_silver.count()

        # === Grava dados Silver ===
        if qtde_registro > 0:
            df_silver.write \
                .mode("append") \
                .jdbc(url=postgres_url, table="db.silver_sptrans.posicao", properties=properties)

        # === Log de sucesso ===
        cur.execute(
            "INSERT INTO db.monitoramento_log.processos_log "
            "(nm_processo, dt_processamento, dt_delta, ds_status, ds_log, qtde_registro) "
            "VALUES (%s, CURRENT_TIMESTAMP, %s, %s, %s, %s)",
            (nm_processo, dt_delta, 1, "Processo executado com sucesso", qtde_registro)
        )
        conn.commit()

    except Exception as e:
        # === Log de falha com mensagem de erro ===
        cur.execute(
            "INSERT INTO db.monitoramento_log.processos_log "
            "(nm_processo, dt_processamento, dt_delta, ds_status, ds_log, qtde_registro) "
            "VALUES (%s, CURRENT_TIMESTAMP, %s, %s, %s, %s)",
            (nm_processo, dt_delta, 0, str(e)[:1000], 0)
        )
        conn.commit()
        print("Erro na conversão Bronze -> Silver:", e)

    finally:
        cur.close()
        conn.close()
        spark.stop()


In [7]:
convert_bronze_to_silver()